# Decision Tree Lab

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from scipy import stats
import math

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits. Otherwise, even with approaches like pruning below, the SSE type of overfit could still hurt us.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [2]:
class Node():
    def __init__(self, name, used, X, y):
        self.name = name
        self.children = []
        self.used = used
        self.split = None
        self.X = X
        self.y = y

class Tree():
    def __init__(self, root):
        self.gains = []
        self.root = root
        
    def dive(self, start, X):
        if len(start.children) > 0:
            ind = None
            for i in range(len(start.children)):
                if start.children[i].name == X[start.split]:
                    ind = i
            if ind is not None:
                return self.dive(start.children[ind], X)
            else:
                return stats.mode(start.y)[0][0]
        else:
            return stats.mode(start.y)[0][0]
    
    def dive_print(self, start, indent):
        if len(start.children) > 0:
            for i in start.children:
                print(indent + "feature " + str(start.split) + " = " + str(i.name))
                self.dive_print(i, indent + "   ")
        else:
            print(indent + "prediction = " + str(stats.mode(start.y)[0][0]))
    
    def add_kids(self, start, gain):
        if np.unique(start.y).shape[0] > 1:
            info_gain = gain(start.X, start.y, start.used)
            if np.amax(info_gain) > 0:
                split = np.argmax(info_gain)
                self.gains.append(info_gain[split])
                start.split = split
                vals = np.unique(start.X[:,split])
                new_used = np.array(start.used)
                new_used[split] = 1
                for i in vals:
                    rows = np.where(start.X[:,split]==i)
                    new_X = None
                    new_y = None
                    for j in rows:
                        if new_X is None:
                            new_X = start.X[j]
                            new_y = start.y[j]
                        else:
                            new_X = np.concatenate((new_X, start.X[j]))
                            new_y = np.concatenate((new_y, start.y[j]))
                    node = Node(i, new_used, new_X, new_y)
                    start.children.append(node)
                    self.add_kids(node, gain)
                    

class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        if counts != None:
            self.counts = counts
        
        else:
            self.counts = None

    def fit(self, X, y):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        
        used = np.zeros(X.shape[1])
        
        root = Node('root', used, X, y)
        
        self.tree = Tree(root)
        self.tree.add_kids(root, self.get_gain)
        
        return self
    
    def get_gain(self, X, y, used):
        
        self.classes = np.unique(y)
        
        class_percents = np.zeros(self.classes.size)
        for i in range(self.classes.size):
            class_percents[i] = np.count_nonzero(y == self.classes[i]) / y.size

        class_info = 0.
        
        for i in class_percents:
            class_info += -i * math.log(i, 2)

        gain = np.zeros(X.shape[1])
            
        for i in range(X.shape[1]):
            if used[i] == 0:
                vals = np.unique(X[:,i])
                for j in vals:
                    at_count = np.count_nonzero(X[:,i]==j)
                    at_percent = at_count / X.shape[0]
                    inf = 0
                    for k in self.classes:
                        tally = 0
                        locations = np.where(X[:,i]==j)[0]
                        for m in range(locations.size):
                            if y[locations[m]] == k:
                                tally += 1

                        if tally > 0:
                            inf += (-tally / at_count) * math.log(tally / at_count, 2)

                    gain[i] += at_percent * inf
                gain[i] = class_info - gain[i]
        
        return gain

    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        
        y = None
        
        for i in X:
            guess = self.tree.dive(self.tree.root, i)
            if y is None:
                y = np.array([guess])
            else:
                y = np.concatenate((y, [guess]))
        return y

    def score(self, X, y):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        
        prediction = self.predict(X)
        
        correct_count = 0
        
        for i in range(len(prediction)):
            if y[i] == prediction[i]:
                correct_count += 1
        
        return correct_count / y.shape[0]



## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff)

Test your model on the lenses test set: [Debug Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv)

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
feature_3 = 0:
	feature_2 = 0:
		feature_0 = 0:
			prediction: 2
		feature_0 = 1:
			feature_1 = 0:
				prediction: 2
			feature_1 = 1:
				prediction: 1
		feature_0 = 2:
			prediction: 2
	feature_2 = 1:
		feature_1 = 0:
			feature_0 = 0:
				prediction: 1
			feature_0 = 1:
				prediction: 1
			feature_0 = 2:
				prediction: 0
		feature_1 = 1:
			prediction: 0
feature_3 = 1:
	prediction: 1
</pre>

In [3]:
# Load debug training data 

data = arff.loadarff('datasets/lenses.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

X = data[:,:-1].astype("U13")
y = np.reshape([data[:,-1]], (-1,1)).astype("U13")

# Train Decision Tree

DT = DTClassifier()
DT.fit(X,y)

# Load debug test data

data = arff.loadarff('datasets/lenses_test.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

test_X = data[:,:-1].astype("U13")
test_y = np.reshape([data[:,-1]], (-1,1)).astype("U13")


# Predict and compute model accuracy

print(DT.score(test_X, test_y))

# Print the information gain of every split you make.

print(DT.tree.gains)
DT.tree.dive_print(DT.tree.root,"")

0.3333333333333333
[0.5487949406953985, 0.7704260414863778, 0.3166890883150208, 1.0, 0.4591479170272448, 0.9182958340544896]
feature 3 = normal
   feature 2 = no
      feature 0 = pre_presbyopi
         prediction = ['soft']
      feature 0 = presbyopic
         feature 1 = hypermetrope
            prediction = ['soft']
         feature 1 = myope
            prediction = ['none']
      feature 0 = young
         prediction = ['soft']
   feature 2 = yes
      feature 1 = hypermetrope
         feature 0 = pre_presbyopi
            prediction = ['none']
         feature 0 = presbyopic
            prediction = ['none']
         feature 0 = young
            prediction = ['hard']
      feature 1 = myope
         prediction = ['hard']
feature 3 = reduced
   prediction = ['none']


In [4]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff)

Test your model on this dataset: [Evaluation Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [5]:
# Load evaluation training data

data = arff.loadarff('datasets/zoo.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

eval_train_X = data[:,:-1].astype("U13")
eval_train_y = np.reshape([data[:,-1]], (-1,1)).astype("U13")

# Train Decision Tree

DTE = DTClassifier()
DTE.fit(eval_train_X, eval_train_y)

# Load evaluation test data

data = arff.loadarff('datasets/zoo_test.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

eval_test_X = data[:,:-1].astype("U13")
eval_test_y = np.reshape([data[:,-1]], (-1,1)).astype("U13")

print(DTE.score(eval_test_X, eval_test_y))

# Print out the information gain for every split you make

print(DTE.tree.gains)

0.147
[1.3630469031539394, 0.8865408928220899, 0.9852281360342516, 0.6962122601251458, 0.8256265261578954, 0.6892019851173655, 0.863120568566631, 0.7219280948873623, 0.7219280948873623]


## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [6]:
# Write a function that implements 10-fold cross validation

def cross_val_10(model, data, results):
    all_data = np.concatenate((data,results), axis=1)
    np.random.shuffle(all_data)
    data = all_data[:,:-1]
    results = all_data[:,-1]
    
    train_acc = []
    test_acc = []
    
    set_size = data.shape[0] // 10
    
    DT = None
    
    for i in range(10):
            train_set = np.concatenate((data[:i*set_size], data[(i+1)* set_size:]))
            train_set_results = np.concatenate((results[:i*set_size], results[(i+1)* set_size:]))
            DT = model()
            DT.fit(train_set, train_set_results)
            train_acc.append(DT.score(train_set, train_set_results))
            test_acc.append(DT.score(data[i*set_size:(i+1)*set_size], results[i*set_size:(i+1)*set_size]))
            
    return train_acc, test_acc, sum(test_acc) / len(test_acc), DT

##  2.2 Cars Dataset
- Use this [Cars Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff)

In [7]:
# Use 10-fold CV on Cars Dataset

data = arff.loadarff('datasets/cars.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

X = data[:,:-1].astype("U13")
y = np.reshape([data[:,-1]], (-1,1)).astype("U13")

train_acc, test_acc, ave_acc, DTC = cross_val_10(DTClassifier, X, y)

# Report Training and Test Classification Accuracies

print(train_acc)
print(test_acc)

# Report Average Test Accuracy

print(ave_acc)

DTC.tree.dive_print(DTC.tree.root, "")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.9011627906976745, 0.9418604651162791, 0.9941860465116279, 0.9534883720930233, 0.9302325581395349, 0.9476744186046512, 0.9534883720930233, 0.9418604651162791, 0.936046511627907, 0.936046511627907]
0.9436046511627907
feature 5 = high
   feature 3 = 2
      prediction = unacc
   feature 3 = 4
      feature 0 = high
         feature 1 = high
            prediction = acc
         feature 1 = low
            prediction = acc
         feature 1 = med
            prediction = acc
         feature 1 = vhigh
            prediction = unacc
      feature 0 = low
         feature 1 = high
            feature 4 = big
               prediction = vgood
            feature 4 = med
               feature 2 = 2
                  prediction = acc
               feature 2 = 3
                  prediction = acc
               feature 2 = 4
                  prediction = vgood
               feature 2 = 5more
                  prediction = vgood
         

## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Note that you will need to support unknown attributes in the voting data set. 

In [8]:
# Used 10-fold CV on Voting Dataset

data = arff.loadarff('datasets/voting_with_missing.arff')
df = pd.DataFrame(data[0])
data = np.array(df)

X = data[:,:-1].astype("U13")
y = np.reshape([data[:,-1]], (-1,1)).astype("U13")

train_acc, test_acc, ave_acc, DTV = cross_val_10(DTClassifier, X, y)

# Report Training and Test Classification Accuracies

print(train_acc)
print(test_acc)

# Report Average Test Accuracy

print(ave_acc)

DTV.tree.dive_print(DTV.tree.root, "")

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.9534883720930233, 0.9767441860465116, 0.9302325581395349, 0.9069767441860465, 0.9302325581395349, 0.8837209302325582, 1.0, 0.8372093023255814, 1.0, 0.9302325581395349]
0.9348837209302324
feature 3 = ?
   feature 11 = ?
      feature 8 = ?
         prediction = republican
      feature 8 = y
         prediction = democrat
   feature 11 = n
      prediction = democrat
   feature 11 = y
      prediction = republican
feature 3 = n
   feature 2 = ?
      prediction = democrat
   feature 2 = n
      feature 11 = ?
         prediction = republican
      feature 11 = n
         feature 5 = ?
            prediction = democrat
         feature 5 = n
            feature 1 = n
               prediction = republican
            feature 1 = y
               prediction = democrat
         feature 5 = y
            prediction = democrat
      feature 11 = y
         prediction = democrat
   feature 2 = y
      prediction = democrat
feature 3 = y
  

## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

Both the cars and the votes got 100% accuracy on all their training sets. I got results in the ranges given above so I am confident that my algorithm is working correctly. Both the cars and the votes got high accuracy which would suggest this is good model for classifying these data sets.

I think that the training gets 100% accuracy because it usually creates a tree where all the leaf nodes are pure (only contain one possible classification). If that doesn't happen, that is probably when the training data doesn't create a tree with pure nodes.

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

The first thing that split the data was the safety rating, the next thing to split it was how many people it carried. If the car had a low safety rating it was always classified as unacc. For the other safety options, (high or medium) if the carry capacity was two it also was classified the car as unacc.

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

The first thing that split the voting data was their opinion on "physician-fee-freeze". If the value of that feature was missing, the next thing to decide on was "export-administration-act-south-africa". If their opinion on that was missing, it would classify the voter as republican otherwise it would say they are democrat. If their opinion on "physician-fee-freeze" was no, more nodes would classify that voter as democrat than republican.

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

During training, my algorithm would treat the missing data as a possible value for the feature, so if there were normally 2 possible values (yes or no) it would also include missing as an option. If my algorithm encountered an unknown attribute value during testing, it treated the node it was on as a leaf. It found what classification was a majority in that node and returned that as its classification. I chose this approach because it was very easy to implement and it has a higher probability of being correct than just picking a random classification from that node.

## 4.1 (10%) Use SciKit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform the best on the test set. 

### 4.1.1 SK Learn on Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)

In [9]:
# Use SK Learn's Decision Tree to learn the voting dataset
for i in range(X.shape[1]):
    le = LabelEncoder()
    le.fit(X[:,i])
    X[:,i] = le.transform(X[:,i])

le = LabelEncoder()
sk_y = np.ravel(y.copy(), order='C')
le.fit(sk_y)
sk_y = le.transform(sk_y)



skDT = DecisionTreeClassifier()
sk_score = cross_val_score(skDT, X, sk_y, cv=10)


# Explore different parameters

# Report results
print("sklearn score\n\n" + str(sk_score))
print("\naverage: " + str(sk_score.sum() / sk_score.shape[0]))
print("\nMy DT score\n\n" + str(test_acc))
print("\naverage: " + str(ave_acc))

skDT = DecisionTreeClassifier(criterion='entropy')
sk_score = cross_val_score(skDT, X, sk_y, cv=10)

print("\nsklearn score with criterion='entropy'\n\n" + str(sk_score))
print("\naverage: " + str(sk_score.sum() / sk_score.shape[0]))

sklearn score

[0.95454545 0.97727273 0.97727273 0.97727273 0.97727273 0.95348837
 0.95348837 0.88372093 0.93023256 0.93023256]

average: 0.9514799154334039

My DT score

[0.9534883720930233, 0.9767441860465116, 0.9302325581395349, 0.9069767441860465, 0.9302325581395349, 0.8837209302325582, 1.0, 0.8372093023255814, 1.0, 0.9302325581395349]

average: 0.9348837209302324

sklearn score with criterion='entropy'

[0.95454545 0.97727273 0.95454545 0.95454545 0.97727273 0.93023256
 0.95348837 0.90697674 0.88372093 0.95348837]

average: 0.9446088794926004


I tried changing the criterion parameter in the sklearn model from default "gini" to "entropy" and the values were very similar.

My algorithm and the sklearn one were usually very close although mine was a few tenths of a percent lower fairly consistently.

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [10]:
# Use SciKit Learn's Decision Tree on a new dataset

df = pd.read_csv('datasets/netflix_titles.csv')
data = np.array(df)

X = data[:,:-1]
y = data[:,-1]

for i in range(X.shape[1]):
    le = LabelEncoder()
    le.fit(X[:,i])
    X[:,i] = le.transform(X[:,i])

le = LabelEncoder()
sk_y = np.ravel(y.copy(), order='C')
le.fit(sk_y)
sk_y = le.transform(sk_y)

skDT = DecisionTreeClassifier()
scores = cross_val_score(skDT, X, sk_y, cv=10)
ave_score = scores.sum() / scores.shape[0]

print(scores)
print(ave_score)

skDT = DecisionTreeClassifier(criterion='entropy')
scores = cross_val_score(skDT, X, sk_y, cv=10)
ave_score = scores.sum() / scores.shape[0]

print(scores)
print(ave_score)

# Experiment with different hyper-parameters

[0.72785623 0.74454429 0.73427471 0.73684211 0.74197689 0.75481386
 0.74550129 0.74550129 0.76735219 0.74550129]
0.7444164128422505
[0.72657253 0.75866496 0.73299101 0.7381258  0.74326059 0.75353017
 0.74678663 0.75064267 0.76863753 0.74550129]
0.7464713181159683


I experimented with the criterion parameter again and I ran this block many times and it seemed like the "entropy" parameter improved the score slightly

## 6. (5%) Visualize your decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

In [11]:
# Include decision tree visualization here

skDT.fit(X, sk_y)

print(export_graphviz(skDT, impurity=False, label='none',max_depth=3))

# Discuss what the model has learned

digraph Tree {
node [shape=box] ;
0 [label="X[2] <= 5.5\n7786\n[5377, 2409]"] ;
1 [label="X[2] <= 2.5\n1424\n[1417, 7]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="X[2] <= 1.5\n126\n[121, 5]"] ;
1 -> 2 ;
3 [label="42\n[42, 0]"] ;
2 -> 3 ;
4 [label="X[0] <= 398.5\n84\n[79, 5]"] ;
2 -> 4 ;
5 [label="(...)"] ;
4 -> 5 ;
6 [label="(...)"] ;
4 -> 6 ;
27 [label="X[0] <= 673.5\n1298\n[1296, 2]"] ;
1 -> 27 ;
28 [label="X[0] <= 335.0\n1290\n[1289, 1]"] ;
27 -> 28 ;
29 [label="(...)"] ;
28 -> 29 ;
32 [label="(...)"] ;
28 -> 32 ;
33 [label="X[2] <= 4.5\n8\n[7, 1]"] ;
27 -> 33 ;
34 [label="(...)"] ;
33 -> 34 ;
35 [label="(...)"] ;
33 -> 35 ;
38 [label="X[0] <= 280.5\n6362\n[3960, 2402]"] ;
0 -> 38 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
39 [label="X[0] <= 212.5\n2124\n[1651, 473]"] ;
38 -> 39 ;
40 [label="X[1] <= 68.5\n955\n[588, 367]"] ;
39 -> 40 ;
41 [label="(...)"] ;
40 -> 41 ;
310 [label="(...)"] ;
40 -> 310 ;
541 [label="X[1] <= 59.5\n1169\n[1063

I honestly can't tell what is going on with this tree display so I don't know what to tell you about what the sklearn algorithm learned

## 7. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).